In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
#load data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

batch_size = 100

batch_count = mnist.train.num_examples // batch_size


def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)


with tf.name_scope('input'):
    #define place holder
    x = tf.placeholder(tf.float32, [None, 784], name='x_input')
    y = tf.placeholder(tf.float32, [None, 10], name='y_input')

with tf.name_scope('layer'):
    with tf.name_scope('weights'):
        #create a 1-level neural network
        W = tf.Variable(tf.zeros([784, 10]), name='W')
        variable_summaries(W)
        
    with tf.name_scope('Biases'):
        b = tf.Variable(tf.zeros([10]), name='B')
        variable_summaries(b)
        
    with tf.name_scope('wx_b'):
        wx_b = tf.matmul(x, W)+b
        
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(wx_b)

with tf.name_scope('loss'):
    #二次代价函数
    # loss = tf.reduce_mean(tf.square(y - prediction))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
    # loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=prediction))
    tf.summary.scalar('loss', loss)

with tf.name_scope('train'):
    #train
    train = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

init = tf.global_variables_initializer()

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_pred'):
        #比较预测值与实际值
        correct_pred = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1)) #argmax 返回沿着某个维度最大值的位置
    with tf.name_scope('accuracy'):
        #准确率
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        tf.summary.scalar('accuracy', accuracy)

#combin all variable monitored
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/', sess.graph)

    for step in range(5000):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        summary, _ = sess.run([merged, train], feed_dict={x:batch_xs, y:batch_ys})
        
        if step % 100 == 0 :
            writer.add_summary(summary, step)
            acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
            print("step:" + str(step) + " accurancy:" + str(acc))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
step:0 accurancy:0.1522
step:100 accurancy:0.7214
step:200 accurancy:0.7996
step:300 accurancy:0.8149
step:400 accurancy:0.8175
step:500 accurancy:0.8227
step:600 accurancy:0.8272
step:700 accurancy:0.8594
step:800 accurancy:0.8731
step:900 accurancy:0.8857
step:1000 accurancy:0.8893
step:1100 accurancy:0.8939
step:1200 accurancy:0.8958
step:1300 accurancy:0.8972
step:1400 accurancy:0.8979
step:1500 accurancy:0.8996
step:1600 accurancy:0.9017
step:1700 accurancy:0.9008
step:1800 accurancy:0.9035
step:1900 accurancy:0.9032
step:2000 accurancy:0.9038
step:2100 accurancy:0.9046
step:2200 accurancy:0.9065
step:2300 accurancy:0.9063
step:2400 accurancy:0.9065
step:2500 accurancy:0.9073
step:2600 accurancy:0.9082
step:2700 accurancy:0.9083
step:2800 accurancy:0.9095
step:2900 accurancy:0.9093
step:300

In [3]:
# with tf.Session() as sess:
#     sess.run(init)
#     writer = tf.summary.FileWriter('logs/', sess.graph)
#     for step in range(1):
#         for batch in range(batch_count):
#             batch_xs, batch_ys = mnist.train.next_batch(batch_size)
#             sess.run(train, feed_dict={x:batch_xs, y:batch_ys})
        
#         acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
#         print("step:" + str(step) + " accurancy:" + str(acc))